**Discrete Event simulation for Take Home 1 of 4DM40**

In [ ]:
# =================================
# Imports
# =================================
from PyCh import *
from numpy import random
import matplotlib.pyplot as plt
matplotlib.use('TkAgg') 


# =================================
# Simulation parameters
# =================================      
filter = "W2"

if filter=="W1":
    ca            = 0.99
    EPTs          =[(43.97, 0.95), (56.34, 0.89), (60.86, 0.9), (62.39, 0.87), (68.3, 0.91), (70.18, 0.9), (72.25, 0.95), (68.15, 0.93), (70.04, 0.95), (76.19, 1.08), (109.77, 0.96), (122.12, 0.75), (11.5, 1.17), (57.5, 0.36)]# list with (te,ce) of wip-dependent EPT-distributions
elif filter=="W2":
    ca            = 0.30                                         # coefficient of variation of inter-arrival time
    EPTs          = [ (461.82, 0.01), (507.0, 0.09), (494.7, 0.1), (531.19, 0.12), (512.94, 0.1), (526.19, 0.11), (529.4, 0.12), (552.3, 0.16), (552.93, 0.12), (564.38, 0.15), (586.93, 0.18), (594.25, 0.19), (597.15, 0.16), (652.31, 0.18), (608.04, 0.16), (629.21, 0.15), (773.8, 0.09), (663.0, 0.0001), (784.0, 0.0001) ]   # list with (te,ce) of wip-dependent EPT-distributions
elif filter=="W3":
    ca            = 3.13
    EPTs          = [(34.31, 0.99), (33.8, 1.0), (33.91, 0.97), (36.35, 0.97), (36.54, 0.93), (34.9, 0.95), (35.48, 0.99), (33.63, 1.01), (35.6, 1.03), (36.16, 1.01), (46.72, 0.82), (50.48, 0.76), (83.83, 0.72), (48.12, 0.77), (140.0, 1.09), (101.5, 0.75), (44.0, 0.0001)]

#
#
warmupcount   = 1000                                         # number of lots to initially ignore (warmup)
lotcount      = 100000                                       # number of lots to run through system (simulation length)
shiftduration = 360.0                                        # duration of shift in minutes
mean_flowtime_result = None

# ====================================================
# Functions and processes for simulation
#
# There should be no need for editing the code below
# ====================================================
def Model():
    """
    Defines and runs the discrete event simulation model
    """
    # define environment
    env = Environment()
    # define channels
    a = Channel(env)
    b = Channel(env)
    # define processes
    G = Generator(env,a)
    W = Workstation(env,a,b,EPTs)
    E = Exit(env,b)
    # run simulation environment
    env.run(until=E)
    
@process
def Generator(env, c_out):
    """
    Generator process:
    Generates lots with mean inter arrival time ta and coefficient of variation ca
    """
    dist = lambda: random.gamma(1/(ca*ca),ca*ca*ta)
    while True:
        x = env.time
        yield env.execute(c_out.send(x))
        delay = dist()
        yield env.timeout(delay)
        
@process
def Workstation(env, c_in, c_out, EPTs):
    """
    Workstation process:
    Consists of a Buffer and a Server.
    
    @param EPTs: List with (te,ce) pairs for wip-dependent EPT-distributions of Server
    @type  EPTs: C{list} of (C{real},C{real})
    
    """
    c_bm = Channel(env)
    B = Buffer(env, c_in, c_bm)
    S = Server(env, c_bm, c_out, EPTs)
    yield B
    yield S
    
@process
def Buffer(env, c_in, c_out):
    """
    Buffer process:
    Adds received lots in list xs, and sends out lots (if available) together with
    the current number of lots remaining in the buffer.
    
    """
    xs = []
    target = 0
    while True:
        sending = c_out.send((xs[0],len(xs)-1)) if len(xs)>0 else None
        receiving = c_in.receive()
        x = yield env.select(sending, receiving)
        if selected(receiving):
            xs = xs + [x]
        if selected(sending):
            xs = xs[1:]
    
@process
def Server(env, c_in, c_out, EPTs):
    """
    Server process:
    Receives lots together with the number of lots in buffer. Based on the latter, selects the EPT-distribution
    to use for determining the processing time, after which the processed lot is sent out.
    If the wip in the workstation is larger than the number of EPT-distributions available in the list, the last
    EPT-distribution is used.
    
    @param EPTs: List with (te,ce) pairs for wip-dependent EPT-distributions of Server
    @type  EPTs: C{list} of (C{real},C{real})
    
    """
    u = [lambda: random.gamma(1/(ca*ca),ca*ca*ta) for (ta,ca) in EPTs]
    maxindex=len(u)-1
    while True:
        (lot,index) = yield env.execute(c_in.receive())
        delay = u[min(index,maxindex)]()
        yield env.timeout(delay)
        yield env.execute(c_out.send(lot))
    
@process
def Exit(env, c_in):
    """
    Exit process:
    Collects all jobs leaving the system and communicates the cumulative production to the controller upon request.
    Also determines the mean flow time of jobs.
    
    """
    n = -warmupcount
    meanflowtime = 0.0
    while n<=lotcount:
        x = yield env.select(c_in.receive())
        n = n + 1
        if n>0:
            flowtime = env.now - x
            meanflowtime = (n-1)/n * meanflowtime + flowtime/n
            
    global mean_flowtime_result
    mean_flowtime_result = meanflowtime
    print(f"Mean flowtime (in minutes): {meanflowtime:f}; mean Throughput (in lots per 6h shift): {n*shiftduration/env.now:f}.")
    
# call Model with required parameters
print(f"Simulation started...")
#Model()

def run_simulation_for_ta(ta_val):
    global ta
    ta = ta_val
    Model()
    return (1 / ta_val, mean_flowtime_result)

if filter=="W2":
    ta_values = [810, 805, 800, 795, 790, 785, 780, 775]
else:
    ta_values = [1400, 1200, 1000, 800, 600, 400, 200, 100, 75, 50, 25]
results = []

print("Running simulations...")
for t in ta_values:
    print(f"ta = {t}")
    results.append(run_simulation_for_ta(t))

# Sort by arrival rate
results.sort()

arrival_rates = [r for r, f in results]
flow_times = [f for r, f in results]

plt.plot(arrival_rates, flow_times, marker='o')
plt.xlabel("Arrival rate (1/ta) [lots/min]")
plt.ylabel("Mean Flow Time [min]")
plt.title(f"Flow Time vs. Arrival Rate for {filter}")
plt.grid(True)
plt.show(block=True)


Simulation started...
Running simulations...
ta = 810
Mean flowtime (in minutes): 1713.474756; mean Throughput (in lots per 6h shift): 0.439967.
ta = 805
Mean flowtime (in minutes): 2081.289813; mean Throughput (in lots per 6h shift): 0.442731.
ta = 800
Mean flowtime (in minutes): 2443.938650; mean Throughput (in lots per 6h shift): 0.445308.
ta = 795
Mean flowtime (in minutes): 3109.468650; mean Throughput (in lots per 6h shift): 0.448402.
ta = 790
Mean flowtime (in minutes): 5763.471770; mean Throughput (in lots per 6h shift): 0.451235.
